In [1]:
#@title Install TF Quant Finance
!pip install tf-quant-finance

In [7]:
#@title Imports
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import functools
import matplotlib.colors as colors

import tf_quant_finance.math.optimizer as tff_optimizer

from matplotlib import pyplot as plt
from IPython.core.pylabtools import figsize

tf.compat.v1.enable_eager_execution()

#tf.enable_eager_execution()
figsize(12, 8)

In [8]:
# Decorator for automatic gradient evaluation.
def make_val_and_grad_fn(value_fn):
  @functools.wraps(value_fn)
  def val_and_grad(x):
    return tfp.math.value_and_gradient(value_fn, x)
  return val_and_grad

In [11]:
# Define a function.
@make_val_and_grad_fn
def func(x):
  return (1 - x[0])**2 + 100*(x[1]-x[0]**2)**2

# Start point for optimization.
start = tf.constant([0.0, 0.0], dtype=tf.float64)

# Optimize
result = tff_optimizer.conjugate_gradient_minimize(func, start)
print("Argmin: %s" % result.position.numpy())
print("Objective value at argmin: %s" % result.objective_value.numpy())


In [10]:
@make_val_and_grad_fn
def func(x):
  return (1 - x[0])**2 + 100*(x[1]-x[0]**2)**2

start = tf.zeros(2, dtype=tf.float64)
delta_range = np.linspace(0.05, 0.5, 9) 
num_iterations = []
num_func_calls = []
for delta in delta_range:
  result = tff_optimizer.conjugate_gradient_minimize(
      func, 
      start, 
      max_iterations=1000,
      params = tff_optimizer.ConjugateGradientParams(
          sufficient_decrease_param=delta))
  num_iterations.append(result.num_iterations.numpy())
  num_func_calls.append(result.num_objective_evaluations.numpy())
  
plt.plot(delta_range, num_iterations, label='Iterations')
plt.plot(delta_range, num_func_calls, label='Objectrive evaluations')
plt.xlabel(r'$\delta$')
plt.legend()
plt.show()

TypeError: Tensor is unhashable if Tensor equality is enabled. Instead, use tensor.experimental_ref() as the key.